# Final Project Submission

Please fill out:
* Student name: Hsin Hin Lim
* Student pace: self paced 
* Scheduled project review date/time: 
* Instructor name: Jeff Herman
* Blog post URL: https://medium.com/swlh/the-map-is-not-the-territory-5024531c9556


# Goal and approach of the project

## Goal of the Project

<div class="alert alert-block alert-success">

For the purposes of this project, let's assume that we are a firm of real estate agents who are new to King County and want to break into its real estate market. In addition to having to learn more about the features of this particular real estate market, the firm will need to be able to act for and advise both potential buyers and sellers as to the price of the property in question.  It needs to be able to generate a benchmark price for any given property as a starting point.

The **goal of this project** is to produce a simple linear model which will produce <b> benchmark predictions for prices </b> that will guide the negotiations between buyers and sellers for a particular property in King County. As a firm of real estate agents who are completely new to the area, having such a benchmark for discussion could prove particularly useful, either to put sales of recent nearby properties into context, or to use as a starting point for negotiations where there is insufficient price data for comparable properties to serve as a meaningful benchmark.
    
While a model can have any number of features, the intention behind limiting the model to 3 features is to find that balance between model complexity and usability; a simple (but accurate) model will be easier for both buyers and sellers to understand and accept as a benchmark for negotiations. </div>

## Approach to the project

The project will move towards that goal in the following 3 broad steps:

1) **Exploratory Data Analysis** - In this step we will look at the entire data set, remove null values, convert data into numerical datatypes, investigate correlations between the data and (crucially) come to a view as to whether certain data is continuous or categorical. Looking at the correlations between the data (and between each independent variable and the dependent variable **price**) will provide initial clues as to which of the independent variables will be good predictors. We will also focus on the distribution of the dependent variable **price** and on the distribution of the independent variables in the dataset. This focus is designed to remove outlier values (of both the dependent and independent variables) which may interfere in the interpretation of any model.

2) **Feature Selection** - Having investigated and cleaned the data in the prior step, we will use a forward selection method to test multiple models to find the 3 features that will give rise to a 3 feature model with the highest **r-squared**. In this step we also take a closer at the features selected and analyse, crucially, whether their residuals are normally distributed.

3) **Assessing model accuracy** - Once we have ascertained that our 3 feature model meets the assumptions of linear regression, we can test the degree of accuracy of the model by analysing the mean error between the price predicted by the model and the actual price data. This will give a sense of the limitations of the model, where the limitations occur and the range of values the actual price of the property can take on away from the benchmark predictions made by the model.

4) Finally, after having identified 3 relevant features - we will use each feature as the basis for asking meaningful questions about the data from the perspective of a firm of real estate agents who are new to King County (as assumed at the outset of our project).

# Exploratory Data Analysis

## Import packages

In [3]:
#import packages
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import cm
plt.style.use('ggplot')

from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import folium
from folium import FeatureGroup, plugins, LayerControl
from itertools import combinations
import scipy.stats as stats

# sklearn packages
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE #RFE is recursive feature elimination to rank features
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score, KFold

# statsmodels packages
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf

# read the data
data = pd.read_csv('kc_house_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'kc_house_data.csv'

## Inspect the Data

In [ ]:
data.info()  

In [ ]:
data.head()

In [ ]:
# generate a table to show where the null values are and how big a problem they might be.
total = data.isnull().sum().sort_values(ascending=False)
percent = ((total/data.isnull().count())*100).sort_values(ascending=False)
missing = pd.DataFrame({'Total':total, 'Percent':percent})
missing

In [ ]:
# unique values

for col in data.columns:
    print(data[col].value_counts())

In [ ]:
# check distribution of values
data.describe()

##  Features which require further exploration/cleaning up

From the above the following variables need to be investigated/cleaned up further:

<div class="alert alert-block alert-info">
    
1. Strings in the **date** column need to be cleaned into a datetime object for easier analysis.

2. Data in **sqft_basement** column is in string format need to be converted into integer format. A dummy variable ('?') also needs to be addressed.

3. **yr_renovated** seems to have a lot of null values which need to be filled in.

4. There appear to be many missing values associated with the **waterfront** variable.

5. There are also missing values associated with the **view** variable.

6. There are 21597 entries but only 21420 *unique* **id**s. This implies some **id**s (and therefore some sale data) as been duplicated in relation to the *same* property.

7. There is a property with **33** bedrooms; this appears to be an outlier, we will remove this from the dataset.</div>

### Converting date to DateTime object

By converting the date from string to DateTime format, we can conduct analysis more easily and even generate a new feature for further analysis by determining the age of each property *at the point of sale*.  The **age** of each property will be determined as the year when the sale took place minus **yr_built** - this should be broadly correct (but may not be precisely so) because the sale data is all from a limited period between 2014 and 2015.

In [ ]:
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%Y') # convert date to a datetime object

data['year_of_sale'] = pd.DatetimeIndex(data['date']).year # extract the year of sale

data['age'] = data['year_of_sale'] - data['yr_built'] # determine the age

In [ ]:
data.age.value_counts()

It is odd that some properties have a *negative* age. These must be new properties which were sold *as they were being built*. Let's correct this.

In [ ]:
data.replace(to_replace=-1, value=0, inplace=True) # replace all negative ages 

### Converting sqft_basement from string into numeric format

In [ ]:
data['sqft_basement'].value_counts()

Since there is only a de minimis quantity of the dummy variable ("?"), we replace the dummy variable with 0 rather than impair the quality of the dataset by dropping the data. 

In [ ]:
data['sqft_basement'].replace(to_replace="?", value=0, inplace=True) # this replaces "?" with a numeric value 0

data['sqft_basement'] = data['sqft_basement'].astype(float) # since all values are numeric now, albeit strings, we can convert to integer

### Investigating and addressing null values and zeros in yr_renovated

#### The null values in yr_renovated

We know there are some null values in the variable **yr_renovated**. Let's create a subset of entries where the **yr_renovated** is null.

In [ ]:
# plot a histogram where the x-axis is the age of a property using the subset of entries where yr_renovated is null.

data.loc[data['yr_renovated'].isnull() == True]['age'].hist()
plt.xlabel('Age of Property')
plt.ylabel('Number of Properties with that age')
plt.show()

#### The zeros in yr_renovated

In addition to the null values, many properties have a zero for **yr_renovated**. Let's plot a histogram of to see the variety of properties with zero as a value for **yr_renovated**.

In [ ]:
# plot a histogram where the x-axis is the age of a property using the subset of entries where yr_renovated is null.

data.loc[data['yr_renovated'] == 0]['age'].hist()
plt.xlabel('Age of Property')
plt.ylabel('Number of Properties with that age')
plt.show()

In [ ]:
data.yr_renovated.value_counts()

Looking at the histogram of properties with zero in yr_renovated and comparing it to the previous histogram which was plotted for properties with null values in yr_renovated, it is reasonable to also assume that such properties have never been renovated. 

Therefore, the strategy should be to fill in all null values with zeros since both null values and zeros indicate that the property has never been renovated.

In [ ]:
data['yr_renovated'].fillna(value=0, inplace=True)

### Waterfront

Null values in relation to **waterfront** is problematic - not only do more than 10% of the entries have a null value in relation to **waterfront** (making it too numerous to drop without impact on overall data quality), it is difficult to impute a value to it since **waterfront** is itself a dummy variable whereby 1 represents a property with *a view to a waterfront* (as stated in the documentation) and 0 represents a property without such view.  Imputing a value of 1 or 0 in this case could have a significant impact on this feature and, in turn, a significant difference to its explanatory value.

The intention behind the following approach is to plot some of the properties onto a map, to see if there is any chance of these properties actually having a view of the water.

First, let's subset the entries with a null value for **waterfront**.

In [ ]:
wf_null = data.loc[data['waterfront'].isnull() == True]
wf_null

There are 2376 such entries, but each of these properties have a different number of floors. It is possible that buildings which are taller will still have a *view* of the waterfront despite not being near the water. Let's subset this further by including only single storey properties. We ought to assume that unless a single storey property is right by the water, it should not have a view of the water. 

In [ ]:
wf_null.floors.value_counts()

In [ ]:
wf_null_single = wf_null.loc[wf_null['floors'] == 1]
wf_null_single

Now let's plot a selection of these 1180 single storey properties on a map, to see if they are anywhere near the water.

In [ ]:
wf_plot_sample = wf_null_single.sample(100) # let's take a sample of 50 single storey properties
lat = wf_plot_sample['lat'].tolist() # generate a list of latitudes corresponding to these 50 properties
long = wf_plot_sample['long'].tolist() # generate a list of longtitudes corresponding to these 50 properties
coordinates = list(zip(lat,long)) # this is a list of coordinates which we will use to plot on folium

In [ ]:
# instantiate map object, using the first on our coordinates list to centre the map.
wf_map= folium.Map(location=coordinates[0], zoom_start=10)

# write a loop to populate the wf_map with markers of single storey properties with null value for waterfront

for c in coordinates:
    folium.Marker(location=c).add_to(wf_map)

wf_map

From a visual inspection of the markers of 100 properties randomly sampled from the subset of single storey properties which have a null value for **waterfront**, it appears that the grand majority (with the exception of maybe 1 or 2) properties are not anywhere close enough to a body of water to actually have a view of the water.  

On this basis, it would be a reasonable assumption to convert all null values, where they relate to *single storey properties* to zero. By doing so we would have also implicitly accepted that multiple storey properties *may* actually have a view of the water; in such cases, the null values would have to be converted to one.

In [ ]:
# this replaces all instances where waterfront contains a null value and the property is single storey with zero
data['waterfront'][(data['waterfront'].isnull() == True) & (data['floors'] == 1.0)] = 0

# this replaces all instances where waterfront contains a null value and the property is multiple storey with one
data['waterfront'][(data['waterfront'].isnull() == True) & (data['floors'] != 1.0)] = 1

data.waterfront.isnull().any() # check for any null values left in waterfront. If False then no null values left.

### Views

There is a minimal number of datapoints with null values in relation to the **view** variable. According to the documentation provided, **view** describes the number of times a property has been viewed by potential buyers. Because the number of datapoints with a null value for this variable are so small(63) in relation to the overall size of the dataset (21000+), entries with a null value can be dropped without unduly affecting the overall quality of the data. 

In [ ]:
## drop the 63 values with null for views.

data.dropna(axis=0, inplace=True, subset=['view'])
len(data) # this proves it is correctly done, with 63 entries less than the 21597 entries we started with

### Duplicate IDs

We had observed that the number of unique IDs was lower than the number of total entries. This means that some entries are repeated. Let's investigate the extent of this problem.

In [ ]:
duplicate_id = data.loc[data.duplicated('id') == True].sort_values(by='id',ascending=True)

In [ ]:
len(duplicate_id)

177 entries are duplicated in the dataset.  This is a relatively small number compared to the overall size of the dataset (21000+). It would therefore be safe to drop the duplicated entries.

In [ ]:
data.drop_duplicates(subset='id', inplace=True)

### Removing the outlier with 33 bedrooms

In [ ]:
data.drop(data[data.bedrooms == 33].index, inplace=True)
data.bedrooms.value_counts()

### Are there any null values left in the dataset?

In [ ]:
data.isnull().any()

No null values appear to be left in the dataset.

### What's left in the dataset after it has been cleaned?

In [ ]:
data.info()

# the following analysis may require us to drop certain data so let's capture a copy of the full set of clean ata at this point

raw_data =data.copy() 

All the data has been transformed into a numeric format that can be used for further analysis.

Also, the dataset now contains **21356** entries - this is correct given that 177 duplicate entries had been dropped, 63 entries dropped for having a null value for **view**, a single entry had been dropped because it had 33 **bedrroms** (therefore regarded as an outlier) and we had started with 21597 entries to begin with. There is some loss of data, but this is minimal compared to the size of the set.

## Feature analysis

### Are the variables categorical, ordinal or continuous?

Let's plot the scatterplot for each of the variables as x-axis (vs the price as the y-axis) to get an idea of its distribution. Depending on the nature of the variable, we may decide to create dummy variables which may improve the output of our linear regression later on.

In [ ]:
for col in data.columns:
    sns.relplot(x= col, y= 'price', data = data)
    

Based on a visual inspection of the scatter plots above showing the distribution of the variables, it is possible to settle on a strategy for how we will treat each of the variables as below:

<div class="alert alert-block alert-info">


<br>1. **id** - this data has no predictive value and can be **dropped** from any feature set or any set that contains independent variables.  <br>
  
<br>2. **date** - this would be the date of sale. The date of sale in the data is either a day in 2014 or 2015, because this is not time series data the date is also of minimal predictive value and can be **dropped** from any set of independent variables.<br> 
  
<br>3. **bedrooms** - the vertical nature of the scatter plot suggests that this is actually a **ordinal** variable.<br>

<br>4. **bathrooms** - the vertical nature of the scatter plot suggests that this is actually a **ordinal** variable. However, the data contains instances of 0.5 or 0.75 a bathroom.<br>

<br>5. **sqft_living** - the cloudlike nature of the scatter plot treated as a **continuous** variable. It even looks like there is a linear relationship with price.<br>

<br>6. **sqft_lot** - the cloudlike nature of the scatter plot treated as a **continuous** variable. By contrast with **sqft_living**, it doesn't look like there is a linear relationship with price. This variable should therefore be **dropped**.<br>

<br>7. **floors** - this is an **ordinal** variable, because there is an order to the number of floors of each property.<br>

<br>8. **waterfront** - this is clearly a **categorical** variable.<br>

<br>9. **view** - this is an **ordinal** variable, the number of times a property is viewed is akin to an ordered score. <br>

<br>10. **condition** - this is a **ordinal** variable because **condition** is an ordered score.

<br>11. **grade** - this is a **ordinal** variable because **grade** is an ordered score.

<br>12. **sqft_above** - this is a **continuous** variable, it even looks like there is a linear relationship with price.<br>

<br>13. **sqft_basement** -this is a **continuous** variable, it does not look like there is a linear relationship with price.  This should therefore be **dropped**. <br>

<br>14. **yr_built** - because the years are ordered, this is an **ordinal** variable.<br>

<br>15. **yr_renovated** - because the years are ordered, this is an **ordinal** variable.<br>

<br>16. **zipcode** - this looks like it may be a **categorical** variable; and because there is no obvious order to the zipcodes, they are **nominal** variables.<br>

<br>17. **lat**,**long** - these are longtitude and latitude for each property. This data may be useful in creating geo-visualisations but, unlike zipcode which may have predictive value, the latitude and longtitude values don't have any predicitve value. These should therefore be **dropped**.

<br>18. **sqft_living15** - the cloudlike nature of the scatter plot treated as a **continuous** variable. It even looks like there is a linear relationship with price.<br>

<br>19. **sqft_lot15** -the cloudlike nature of the scatter plot treated as a **continuous** variable. By contrast with **sqft_living15**, it doesn't look like there  is a linear relationship with price.  This should therefore be **dropped**.<br>

<br>20. **year_of_sale** - as before, all years of sale in this dataset are 2014 and 2015 and because it is not a time series, this variable would have little predictive value. This should therefore be **dropped**.

<br>21. **age** - because there is order to the age of the property (in years), this is an **ordinal** variable. <br>

</div>

In [ ]:
# let's explicitly declare the continuous features

continuous = ['sqft_living','sqft_living15','sqft_lot','sqft_lot15','sqft_basement', 'sqft_above']

# separate df of continuous features
continuous_df = data.loc[:, continuous]

continuous_df.columns

In [ ]:
# let's explicitly declare ordinal and categorical features

ordinal = ['bedrooms', 'bathrooms','floors','view', 'condition', 'grade','yr_renovated']
categorical = ['waterfront','zipcode']

non_continuous = ordinal + categorical

# the following loop turns the categorical data into the category datatype.
for cat in categorical:
    data[cat] = data[cat].astype('category') 

In [ ]:
# change price from float to integer

data['price'] = data['price'].astype(int)

In [ ]:
# dropping the features identified above
data.drop(['id','date','sqft_lot','sqft_basement','lat','long','sqft_lot15','year_of_sale'],axis=1, inplace=True)

In [ ]:
data.info()



### Which non-continuous variables have a linear relationship with price?

We can tell from the scatterplots of **sqft_living** and **sqft_above** that there is a linear relationship between those features when plotted against price. We can equally test whether there is a linear relationship between price and each **non-continuous** variable by using boxplots. 

In [ ]:
for cat in non_continuous:
    fig, ax = plt.subplots()
    fig.set_size_inches(11.7,8.27)
    ax = sns.boxplot(x=data[cat], y = data['price'])
    ax.set_title(f'{cat} vs price')
    ax;

Based on an analysis of the boxplots of each categorical variable above, we come to the following conclusions in relation to each of the categorical features:

<div class="alert alert-block alert-info">

<br>1. **bedrooms** - there appears to be a linear relationship with price, but it is a weak one. <br>

<br>2. **bathrooms** - again there appears to be a linear relationship with price, but in contrast with bedrooms, it is a lot stronger. At this point, we could consider including bathrooms as a feature in the model.<br>

<br>3. **floors** - there appears to be a limited, and almost no, linear relationship with price.This should be **dropped**.<br>

<br>4. **waterfront** - There doesn't appear to be any linear relationship between price and this feature. This should be **dropped**.<br>

<br>5. **view** - there is a linear relationship with price, but it appears to be a weak relationship. We could consider including this feature in our model. <br>

<br>6. **condition** - There doesn't appear to be any linear relationship between price and this feature.This should be **dropped**.

<br>7. **grade** - There is a very clear relationship between price and this feature. In fact, one might be able to plot a curve that describes this relationship. This relationship may not be linear, but polynomial.

<br>8. **age** - There is no real relationship between this feature and price. This should be **dropped**.<br>

<br>9. **yr_renovated** - There does not look like any substantial relationship between this feature and price. In fact, the grand majority of observations in this variable are zeros(a zero implies the property has never been renovated). Looking at the box plot; the very noticeable tail of outliers when this value is zero (at the very leftmost of the plot) further implies that there is no clear pattern between whether (and when) a property has been renovated. Therefore, this should be **dropped**. <br>

<br>10. **zipcode** - The distribution looks inconclusive. On the basis of conventional wisdom, which dictates that the location of a property is one of the main drivers of its value, we should retain this feature for now. We should make dummy variables out of this feature to determine whether there is a linear relationship.<br>
</div>

In [ ]:
# dropping the features identified above
data.drop(['floors','waterfront','condition','age','yr_renovated'],axis=1, inplace=True)

In [ ]:
# check which features are left in data
data.columns

### What is the distribution of the dependent variable?

Let's identfy if any of the property prices observed are outliers. We may want to remove outlier prices from the dataset to improve any interpretation we may subsequently make of the fit.  It appears from the plot that there is a rather long tale of outlier values that could interfere with the interpretation of any fit we make.

In [ ]:
sns.distplot(data['price']);    

By working out where the 10th and 90th quantile in prices, we can eliminate outlier in prices. On that basis, we discover that we ought to limit the dataset to properties with prices between (approximately) **USD250,000** and **USD 890,000**; this should remove outliers in prices.

In [ ]:
data['price'].quantile([0.10,0.90])

### Identifying high leverage predictors from the scatter plot

From the scatterplots above, we are also able to identify features which have observations which are large relative to the other observations.  For instance, in relation to **sqft_living**, properties which are 6,000 sqft and above have living space which far exceeds those of the other properties in the data set. The intention should be to remove these observations from the dataset to improve the fit of the model. 

Other variables with high leverage observations include:

<br>1) **Bathrooms** (where properties with more than 6 bathrooms should be excluded) 
<br>2) **sqft_above** (where properties with more than 6,000 sqft should be excluded)
<br>3) **sqft_living15** (where properties with more than 5,000 sqft should be excluded)

### Addressing high leverage predictors and outliers in the distribution of price

Given the analysis above, the intention is to **drop** properties with:

<br>1) prices that are less than USD 250,000 and more than USD 890,000
<br>2) **sqft_living** or **sqft_above** more than 6,000 sqft
<br>3) **sqft_living15** more than 5,000 sqft
<br>4) 6 or more **bathrooms**.

The following is intended to remove properties with outlier prices.

In [ ]:
data = data[(data['price']>250000) & (data['price']<890000)]
sns.distplot(data['price']);

The price data is positively skewed.

The following is intended to remove predictors with high leverage. We first set out the criteria for the observations to work with which will remove the high leverage predictors before using the criteria to select the data we want to work with.

In [ ]:
data.info()

In [ ]:
criteria = (data['sqft_living'] < 6000) & (data['sqft_above'] < 6000) & (data['sqft_living15'] < 5000) & (data['bathrooms'] < 5)
data = data[criteria]

In [ ]:
data.info() # we are left with 16824 observations

This leaves us with **16824** observations in the dataset.

### Heatmap showing the correlation between features and price

Let's have a quick look at the correlation between the features we have retained after the analysis above, noting in particular, the correlation between price and the other features (except for the categorical feature **zipcode**). This will provide some initial ideas as to the explanatory value of each feature in determining the price.

In [ ]:
# plot sns heatmap of feature correlations
plt.figure(figsize=(20,15))
plt.title('Initial Exploration of Correlations between Features from dataset')
hmap = sns.heatmap(data.corr(), center=0, annot=True, cmap='PiYG')
hmap.set_xticklabels(hmap.get_xticklabels(), rotation=45, horizontalalignment='right')
hmap.set_yticklabels(hmap.get_yticklabels(), rotation=45, horizontalalignment='right');

In [ ]:
# correlation with price
corr_matrix = data.corr()
target_corr = abs(corr_matrix['price'])

# this will show us the features with correlation >0.5
target_corr[target_corr > 0.5]

From both a visual inspection of the heatmap and the code above, we have some idea of the variables which display a strong correlation to price. The following variables are those that display a higher than 0.5 correlation with price:

1. **sqft_living**
2. **grade**

That is not to eliminate the other variables from being strongly correlated with price. This is particularly so with regard to categorical variables like **zipcode** which, from everyday experience, tells us that the location of a property is one of the main drivers in price. 

In addition, of the 5 variables identified above it is worth pointing out that **sqft_living** and **sqft_above** are highly correlated (0.88). This is unsurprising; a priori, a property with a large living space (described sqft_living) will imply that the property has a large living space apart from the basement (described by sqft_above), if any. Any model would probably pick one of **sqft_above** or **sqft_living** as a consequence of avoiding multicollinearity.

It is also a key requirement of the project that any predictor chosen must have a p value of less than 0.05 - this has not been checked yet for any of the variables listed above.

### Heatmap showing potential multicollinearity between features

As a final step of the EDA, let's look at the correlation matrix between all features (and not just the correlation between price and the remaining features). Let's assume that any correlation higher than 0.75 will indicate strong correlation between features - this isn't ideal because linear regression works on the basis that independent variables are not correlated. The heatmap will help us identify with which pairs are problematic.

In [ ]:
# let's run a heatmap again, but this time with correlations more than 0.75 highlighted
plt.figure(figsize=(20,15))
plt.title('Initial Exploration of Potential Multicollinearity between Features')
hmap = sns.heatmap((abs(data.corr()) > .75), center=0, cmap='PiYG', linewidths=1.5)
hmap.set_xticklabels(hmap.get_xticklabels(), rotation=45, horizontalalignment='right');

These pairs indicate strong possibility of multicollinearity between sqft_living vs sqft_above.

The above results suggest that when creating the feature set, we should drop either **sqft_living** or **sqft_above**. Let's drop **sqft_above** with the following line of code:

In [ ]:
data.drop(['sqft_above'],axis=1, inplace=True)

## Conclusions

In [ ]:
data.info()

<div class="alert alert-block alert-info">
Summarising the insights above, it looks like that at this stage, our model ought to include some of the following features:

<br>1. **bedrooms**
<br>2. **bathrooms**
<br>3. **sqft_living**
<br>4. **view**
<br>5. **grade**
<br>6. **yr_built**
<br>8. **zipcode**
<br>9. **sqft_living15**

We retain **zipcode** at this stage because we intend to encode dummy variables out of them to test whether they are good predictors of price. 
</div>

# Creating a feature set, training set and test set

## The feature set

Let's drop the variables specified during the process of the EDA to create a features matrix:

In [ ]:
features = data.copy().drop(['price'], axis=1)
target = data['price']
features.info()

In [ ]:
data['view'].value_counts()

## Dummy variables

Create dummy variables in relation to **zipcode**.

In [ ]:
# converting zipcode

zipcode_dummy = pd.get_dummies(features.zipcode, drop_first=True, prefix='zip')
zipcode_dummy

## Joining up the dummy variables to the other features

In [ ]:
features_dummies = pd.concat([features,zipcode_dummy], axis=1).drop(['zipcode'], axis=1)
features_dummies

## Creating a test set and a training set

### Test and training set for use with Statsmodels

This next line of code is important because it sets up the training and test sets. The intention is to fit the model using the training set and use the test set to gauge the predictive value of that model.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=42)

In [ ]:
X_train.head()

In [ ]:
X_train.isnull().any().any() # double check if there are any null values in X_train which would prevent regression.

And so, going forward from this point, the intention is to fit the model using **X_train** and **y_train**. Once this is accomplished we will check the predictive ability of the model against **X_test** and **y_test** where we use Statsmodels. For convenience, we will constitute a **training_data** dataframe which will combine the target variable back together with the features except this dataframe only covers 75% of the available data to work with as a training set.

In [ ]:
training_data = pd.concat([X_train,y_train], axis = 1) # this is the training data on which to fit the model

### Test and training set for use with Scikit Learn

The next line of code is also to generate a training and test set, but this time these sets (suffixed with *dummy*) is for use with SciKit Learn.

In [ ]:
X_train_dummy, X_test_dummy, y_train_dummy, y_test_dummy = train_test_split(features_dummies, target, test_size=0.25, random_state=42)

In [ ]:
X_train_dummy.head()

# Feature Selection

It's more flexible to use Statsmodels to first assess the R-squared of different models by varying the different features for inclusion an R-style patsy. In addition to comparing different models by their R-squared, we can also use the Statsmodels library to:

<br>1) test the key model requirement that the p-value of each feature must be less than 0.05;
<br>2) generate QQplots to test whether the errors of any given model is normally distributed;

Once we have found a suitable model, we shall then use the LinearRegression method in SciKit-Learn to fit a model using the same features identified in the process of using Statsmodels to compare different models. We shall make an initial start by adopting a forward selection method that will pick out a model with 3 features.

## Forward selection of features

Assuming that we want to restrict ourselves to a model with only 3 features to balance model complexity with accuracy, which three features do we pick? Let's write a simple function that picks out the best 3 features that will maximise the r-squared of a model with 3 features.

In [ ]:
def fwd_select_r(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by R-squared
    
    stopping rule: when 3 features have been selected
    """
    remaining = list(data.columns)
    remaining.remove(response)
    selected = []
    while len(selected) < 3:
        scores_with_candidates = []
        for candidate in remaining:
                        
            # next 2 lines model the features
            formula = "{target} ~ {patsy} + 1".format(target=response,patsy='+'.join(selected + [candidate]))
            model = ols(formula=formula,data = data).fit()               
            
            
            # scoring the model in question based on its mean squared error; the lower the mse the better
            current_score = round(model.rsquared,3)
            
            # each model is distinguished by its set of features, this sets out a list of scores
            # and the corresponding model (each distinguished by its features)
            scores_with_candidates.append((current_score, candidate)) 
            scores_with_candidates.sort()                                               
            
        # because we have sorted out the list of score/model, the best score/model is the last in the list
        # we can then pop it out and assign it to best_new_score/best_model
        best_score, best_model = scores_with_candidates.pop()
        selected.append(best_model)
        remaining.remove(best_model)
        
        print(f'Features selected: {selected}')
       
        print(f'Features remaining: {remaining}')
        # if the mse for the current model is higher than the best score then that model isn't very good
        # we can then discard the feature added to the previous features to get to that model
        # otherwise, the mse for the current model will be lower, that means we can add the feature added
        # to the list of features which are selected
        
    # the 2 lines of code just fits a model based on those that are selected
    formula = "{target} ~ {patsy} + 1".format(target=response,patsy=' + '.join(selected))
    model = ols(formula, data).fit()
    final_score = round(model.rsquared,3)
    return final_score, selected

Now, let's apply the function to **training_data**, the dataframe which retains only the features left over from our EDA above, when we eliminated certain features due to their collinearity with other features or because they did not exhibit a linear relationship with price.

In [ ]:
model = fwd_select_r(training_data, 'price')
model # through the print statements in the function, we can see the selection algorithm work

Our initial observation is that our model should have 3 variables, **zipcode**, **sqft_living** and **grade** to provide an r-squared of **0.734**. This makes intuitive sense since the location, size and quality of the property are typically the uppermost considerations when purchasing a property. We need to analyse this model further to investigate whether the assumptions of linear regression are satisfied. Let's start by using these 3 features to fit a model, obtain the model diagnostics, and ascertain whether each feature has a p-value of less than 0.05 (as per the requirement).

In [ ]:
# this adds a constant before statsmodel linear regression
# we only need to do this once and we can use training _data for the remaining analysis
training_data = sm.add_constant(training_data, prepend=False) 

formula = 'price ~ C(zipcode) + sqft_living + grade' #this is the model

model = ols(formula=formula,data = training_data).fit() # this fits the model to the training data.
model.summary()

It appears **sqft_living** and **grade** have very small p-values that are lower than 0.05. It would appear at this stage that they are valid as features. It is unclear whether the same can be said of **zipcode**; because we have modelled it as a categorical variable, we cannot tell from the diagnostics whether **zipcode** as a category has a p-value of less than 0.05.

Because we cannot obtain a p-value for **zipcode** as a category, we use an F-test instead by generating an ANOVA table.

In [ ]:
table = sm.stats.anova_lm(model, typ=2)
baseline_anova_df = pd.DataFrame(table)
baseline_anova_df

Based on the ANOVA table above, the p-value for **zipcode** (and in fact the p-value for **sqft_living** and **grade**) are less than 0.05. As part of an F-test, this result allows us to reject the null hypothesis that all of the regression coeeficients in our model are equal to zero. That is to say, we can reject the hyothesis that our model has no predicitive capacity and accept the alternative hypothesis that our model has a predictive power that is unlikely to be due to luck.

## Analysis of residuals relating to each feature

### zipcode

Let's fit **zipcode** as a single feature to a model and visualise the distribution of its residuals.

In [ ]:
formula = 'price ~ C(zipcode)' #this is the model

zip_model = ols(formula=formula,data = training_data).fit() # this fits the model to the training data.
residual = zip_model.resid
fig = sm.graphics.qqplot(residual, dist=stats.norm, line='45', fit=True)

The residuals (in blue) very nearly track the red 45 degree line that denotes a normal distribution. This means the residuals as relates to **zipcode** are very close to being normally distributed. This suggests **zipcode** is a valid feature which we can include in our model.

### sqft_living

Let's fit **sqft_living** as a single feature to a model and visualise the distribution of its residuals.

In [ ]:
formula = 'price ~ sqft_living' #this is the model using only sqft_living as a feature

sqft_model = ols(formula=formula,data = training_data).fit() # this fits the model to the training data.
residual = sqft_model.resid
fig = sm.graphics.qqplot(residual, dist=stats.norm, line='45', fit=True)

Like in the case of **zipcode**, the residuals in the case of **sqft_living** look very close to being normally distributed. This suggests that **sqft_living** is a valid feature we can include in our model.

### grade

In [ ]:
formula = 'price ~ grade' #this is the model using only grade as a feature

grade_model = ols(formula=formula,data = training_data).fit() # this fits the model to the training data.
residual = grade_model.resid
fig = sm.graphics.qqplot(residual, dist=stats.norm, line='45', fit=True)

Again, it appears that the residuals as they relate to **grade** look very close to being normally distributed.  This suggests that **grade** is also a valid feature we can include in our model. 

### QQplot of residuals of final model

In [ ]:
formula = 'price ~ C(zipcode) + sqft_living + grade' #this is the model using only grade as a feature

grade_model = ols(formula=formula,data = training_data).fit() # this fits the model to the training data.
residual = grade_model.resid
fig = sm.graphics.qqplot(residual, dist=stats.norm, line='45', fit=True)

Thus from a QQplot of the final 3 feature model, it appears that the residuals are more normally distributed around the mid range of price values and that the model will be more accurate around that range.

## Explaining the final 3 feature model

<div class="alert alert-block alert-info">

At this point, we can settle on a suitable 3 feature model being a model which includes:
    <br>1. <b>zipcode</b>
    <br>2. <b>grade</b>
    <br>3. <b>sqft_living</b>
    
It might also be worthwhile recalling that when we had previously plotted the correlation between price and each feature in a heatmap, **grade** and **sqft_living** stood out for their high correlation with price. In addition, these two features also had a clear linear relationship with price when we investigated whether certain continuous (and non-continous) features had a linear relationship with price.
    
Intuitively this makes sense; price of a property ought to increase and decrease with the quality of the property. It also makes sense that one ought to pay more (or less) for more (or less) living space. 

Lastly, applying everyday experience, location is a major (if not the primary) determinant of a property's price - it is in fact the only thing about a property that one cannot change. It is possible to repair a property to improve its quality. It is even possible (though expensive and maybe bureaucratically tricky, but possible) to expand the living space of a property, but a property's location is something intrinsic to a property and therefore ought to be a key driver to its value. In our dataset, the property's **zipcode** is a proxy for its location and therefore a key determinant of tis value.

</div>

# Assessing the model

## What does the model look like?

We need to judge our model using the test set split out earlier. We shall use **X_train_dummy** as a starting point because it already contains the dummy variables in relation to **zipcode**.  We can then obtain a **final_model_train** by retaining **sqft_living**, **grade** and the dummy variables in relation of **zipcode** while dropping the other columns from **X_train_dummy**. We will then use **final_model_train** and **y_train** to train our model from this point.

In [ ]:
final_model_train = X_train_dummy.drop(['bedrooms','bathrooms','view','yr_built','sqft_living15'],axis=1)

Equally, we should create a **final_model_test**. We shall use **final_model_test** and **y_test** to assess the quality of our model.

In [ ]:
final_model_test = X_test_dummy.drop(['bedrooms','bathrooms','view','yr_built','sqft_living15'],axis=1)

Let's fit the model using **final_model_train** and by using only the values of the indepedent variables which relate to the test set (**y_train_dummy**)

In [ ]:
# train the linear regression model using final_model
linreg = LinearRegression()
linreg.fit(final_model_train,y_train_dummy)

Because of the dummy variables that come from hot encoding **zipcode**, there are **71** coefficients in our model. See the below for the value of each coefficient and intercept.

In [ ]:
coefficients,intercept = linreg.coef_, linreg.intercept_
model_dict = dict(zip(final_model_train.columns,coefficients))
model_dict

## Assessing the residuals of the model

Using the YellowBrick library, we can generate a residual plot such that we see the difference between the actual prices and the predicted prices.  The histogram to the side of the main plot indicates that the residuals are normally distributed around zero, which in turn indicates an unbiased model.

In [ ]:

vis = ResidualsPlot(linreg,size=(1080, 720))

vis.fit(final_model_train, y_train) # fit the training data to the visualiser
vis.score(final_model_test, y_test) # Evaluate the model on the test data
vis.show();

## Assessing the accuracy of the predictions

Again, using the YellowBrick library, we use a prediction error plot to show the actual prices from the test set against the predicted prices generated by our model. 

In [ ]:
visualiser = PredictionError(linreg,size=(1080, 720),bestfit=True,line_color = 'red')
visualiser.fit(final_model_train, y_train) # fit the training data to the visualiser
visualiser.score(final_model_test, y_test) # Evaluate the model on the test data
visualiser.show();

Judging by the plot, our model over-predicts prices for properties that are on the cheaper end of the scale but under-predicts properties that are on the more expensive end of the scale.

## Quantifying the error in prediction using cross validation & mean squared error

In [ ]:
# cross validation using 5 folds and coming up with the average mean squared error across all 5 sets
linreg = LinearRegression()
mse = make_scorer(mean_squared_error)
cross_validation = KFold(n_splits=5, shuffle=True, random_state=1)
cv_5_results = cross_val_score(linreg, final_model_test, y_test_dummy, cv=cross_validation, scoring=mse)
round(cv_5_results.mean(),6)**0.5

The average mean square error is **85,025**. This means the prediction generated by our 3 feature model is, on average, off the actual price by approximately **USD85,000**

## Deploying the model

Let's assume that buyer and seller of a property in zipcode 98014, with 1,500 sqft of living space and built to grade 9 want to know the benchmark from which they should begin negotiations for the sale of the property. At what price should they start?

In [ ]:
# write a function which takes sqftliving, grade and zipcode (as a string, in the format 'zip_98014') as arguments
# the function then returns the benchmark price which can be used as the starting point of negotiations

def benchmark_price(sqft_living,grade,zipcode):
    bench = (sqft_living*model_dict['sqft_living']) + (grade*model_dict['grade']) + model_dict[zipcode] + intercept
    return round(bench,2)

Let's assume a hypothetical property that has 1,500 sqft in living area, built to a grade 9 standard and located in zipcode 98014. 

In [ ]:
benchmark_price(1500,9,'zip_98014')

Our model suggests a benchmark price of **USD461,507** that buyers and seller can use as a potential starting point for negotiations. 

## Conclusions and practical application of model output

<div class="alert alert-block alert-info">

Recall that the goal of this project is to provide predictions for the price of a specific property in King County such that our firm has a benchmark for negotiations between buyers and sellers. The model is simple to understand for both buyers and sellers and therefore more likely to be accepted as the basis for negotiations, but the model is not perfect. In the negotiations, both buyers and sellers should be aware that the output of the model will tend over predict prices for cheaper properties and tend to under predict prices for more expensive properties.

The firm of real estate agents using this model should also be aware that the price predicted by the model could be wrong **on average** by USD 85,000. This does not mean that the price predicted is always wrong by that amount; it appears that the more expensive the property is, the error grows larger. It also does not mean that the model is useless for its stated purpose (to provide a benchmark for negotiations). It just means that in using the predicted prices as a benchmark, parties have to factor the average error into their negotiations.

</div>

# Asking further meaningful questions of the King County dataset

## Analysing the data by zipcode

### Looking at the distribution of the features in our model

<div class="alert alert-block alert-info">Recall our starting assumption at the beginning of the project - we are a firm of real estate agents who are new to King County. We are therefore not very familiar with the terrain, but let's try to imagine the residential landscape through the data by asking a few <b>meaningful questions</b>:
<br>
    <ol>
        <li> Where are the oldest neighbourhoods?</li>
 <li>Which areas have "fancy" housing? This is likely to also include the most expensive neighbourhood but is also likely to include other "nice" areas.</li>
 <li>Where does one tend to find the large houses in King County?</li><ol>
</div>

We can answer these meaningful questions by analysing the geographical distribution ( by **zipcode**) of the median values of  **grade**, **sqft_living** and **age** of property across King County.  We shall do so by visualising the median values onto a **choropleth map**. First, we want to prepare the data by importing and cleaning the geojson file of the relevant **zipcode** boundaries into this notebook.

In [ ]:
#load geojson downloaded from data.settle.gov

with open('zip_codes.geojson','r') as file:
    tmp = json.load(file)

# the following counts how many unique zipcodes there are from the dataset post-clean up   
data.zipcode.nunique() 

# coords are the coordinates of each data point
# this is taken from the dataset pre-clean up
lat = raw_data['lat']
lon = raw_data['long']
coords = list(zip(lat,lon))

# create a list with all the unique zipcodes in the dataset

data_zips = list(set(raw_data.zipcode))
len(data_zips) # this is correct, there are 70 unique zipcodes in the dataset

# create a list geozips with from the zipcodes IN THE JSON that are ALSO in data_zips

geozips = list(set([int(tmp['features'][x]['properties']['ZIPCODE']) for x in range(len(tmp['features'])) if int(tmp['features'][x]['properties']['ZIPCODE']) in data_zips]))
len(geozips) # this is correct; there are 70 unique zipcodes in data_zips. There are 70 unique zipcodes in geozips

# remove zipcodes not in our dataset

req_zip = [tmp['features'][x] for x in range(len(tmp['features'])) if int(tmp['features'][x]['properties']['ZIPCODE']) in geozips]

# create a new JSON object
new_json = dict.fromkeys(['type','features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = req_zip

# save the new JSON object as updated_json
open('updated_json.json','w').write(json.dumps(new_json, sort_keys=True, separators=(',',':')))


Now we want to use the geojson file and our dataset to create the visualisation.

In [ ]:
# instantiate map
mp = folium.Map(location=coords[777], zoom_start=10, tiles='cartodbpositron')

# data and choropleths for median grade by zipcode
table = pd.DataFrame(raw_data.groupby('zipcode')['grade'].median()).reset_index()
choropleth =  folium.Choropleth(geo_data='updated_json.json', data=table, 
              columns = ['zipcode','grade'], 
              key_on= 'feature.properties.ZIP',
              bins = 6,
              fill_color= 'YlGnBu', 
              fill_opacity = 0.5,
              line_opacity = 0.3,
              line_color= 'black',
              legend_name = 'Median Grade',
              name = 'Median Grade',
              show=False,
              highlight= True).add_to(mp)
choropleth.geojson.add_child(folium.features.GeoJsonTooltip(['ZIPCODE'], labels=True))

# data and choropleths for median sqft_living by zipcode
table_2 = pd.DataFrame(raw_data.groupby('zipcode')['sqft_living'].median()).reset_index()
choropleth_2 =  folium.Choropleth(geo_data='updated_json.json', data=table_2, 
              columns = ['zipcode','sqft_living'], 
              key_on= 'feature.properties.ZIP',
              bins = 6,
              fill_color= 'YlGnBu', 
              fill_opacity = 0.5,
              line_opacity = 0.3,
              line_color= 'black',
              legend_name = 'Median Sqft_living',
              name = 'Median Sqft_living',
              show=False,
              highlight= True).add_to(mp)
choropleth_2.geojson.add_child(folium.features.GeoJsonTooltip(['ZIPCODE'], labels=True))

# data and choropleths for median age of property by zipcode
table_3 = pd.DataFrame(raw_data.groupby('zipcode')['age'].median()).reset_index()
choropleth_3 =  folium.Choropleth(geo_data='updated_json.json', data=table_3, 
              columns = ['zipcode','age'], 
              key_on= 'feature.properties.ZIP',
              bins = 6,
              fill_color= 'YlGnBu', 
              fill_opacity = 0.5,
              line_opacity = 0.3,
              line_color= 'black',
              legend_name = 'Median Age',
              name = 'Median Age',
              show=False,
              highlight= True).add_to(mp)
choropleth_3.geojson.add_child(folium.features.GeoJsonTooltip(['ZIPCODE'], labels=True))

LayerControl(position='bottomright', collapsed=False).add_to(mp)

mp.save('KC_map.html')
mp

<div class="alert alert-block alert-info">
    The choropleth map above allows users to overlay median <b>age</b>, <b>grade</b>, <b>sqft_living</b> over a map of King County. By doing so, we can obtain the following insights (though I have never been to King County):    
<ol>
    <li> the older neighbourhoods (by <b>age</b>) are clustered around zipcodes 98102, 98103, 98105 and 98109;</li>
    <li> the fancier neighbourhoods (by <b>grade</b>) in King County are clustered around zipcodes 98039,98040, 98004 and 98006. There is also a further enclave of "nice" housing around zipcodes 98074, 98075 and again to the north at zipcode 90877; and</li>
    <li> some of the zipcodes mentioned above also have the larger properties (by <b>sqft_living</b>), but it's worth pointing out that zipcode 98040 also stands out as an area with large residential housing.</li>
    <ol>
</div>

### Looking at the sales data by zipcode

<div class="alert alert-block alert-info">What should be the firm's sale strategy? It may not be possible for the firm to cover all of King County initially; it may need to target some areas first but the areas it targets could depend on the sales strategy it wants to follow. There are two possible strategies to maximise profit:    
<br>
<ol>
<li> Target as many sales as possible - a <b>volume strategy</b>. Therefore, I will need to know which <b>zipcodes</b> have seen the highest number of sales in my dataset.</li>
<li> Target high value sales - a <b>value strategy</b>. Therefore, I will need to know which <b>zipcodes</b> have the highest median <b>price</b> (something I already know from the section above). I will also need to know which <b>zipcodes</b> have seen the highest amount of sales by value. </li>
    </ol>
</div>

Again, we can use a chloropleth map as above to answer these questions:

In [ ]:
# instantiate map
mp_sales = folium.Map(location=coords[777], zoom_start=10, tiles='cartodbpositron')

# data and choropleths for total number of sales by zipcode
table_1_sales = pd.DataFrame(raw_data.zipcode.value_counts().reset_index())
choropleth_sales_1 =  folium.Choropleth(geo_data='updated_json.json', data=table_1_sales, 
              columns = ['index','zipcode'], 
              key_on= 'feature.properties.ZIP',
              bins = 6,
              fill_color= 'YlOrRd', 
              fill_opacity = 0.5,
              line_opacity = 0.3,
              line_color= 'black',
              legend_name = 'Total Number of Sales',
              name = 'Total Number of Sales',
              show=False,
              highlight= True).add_to(mp_sales)
choropleth_sales_1.geojson.add_child(folium.features.GeoJsonTooltip(['ZIPCODE'], labels=True))

# data and choropleths for total value of sales by zipcode
table_2_sales = pd.DataFrame(raw_data.groupby('zipcode')['price'].sum()).reset_index()
choropleth_sales_2 =  folium.Choropleth(geo_data='updated_json.json', data=table_2_sales, 
              columns = ['zipcode','price'], 
              key_on= 'feature.properties.ZIP',
              bins = 6,
              fill_color= 'YlOrRd', 
              fill_opacity = 0.5,
              line_opacity = 0.3,
              line_color= 'black',
              legend_name = 'Total Value of Sales',
              name = 'Total Value of Sales',
              show=False,
              highlight= True).add_to(mp_sales)
choropleth_sales_2.geojson.add_child(folium.features.GeoJsonTooltip(['ZIPCODE'], labels=True))

# data and choropleth for median prices by zipcode
table_3_sales = pd.DataFrame(raw_data.groupby('zipcode')['price'].median()).reset_index()
choropleth_sales_3 =  folium.Choropleth(geo_data='updated_json.json', data=table_3_sales, 
              columns = ['zipcode','price'], 
              key_on= 'feature.properties.ZIP',
              bins = 6,
              fill_color= 'YlOrRd', 
              fill_opacity = 0.5,
              line_opacity = 0.3,
              line_color= 'black',
              legend_name = 'Median Prices',
              name = 'Median Prices',
              show=False,
              highlight= True).add_to(mp_sales)

choropleth_sales_3.geojson.add_child(folium.features.GeoJsonTooltip(['ZIPCODE'], labels=True))


LayerControl(position='bottomright', collapsed=False).add_to(mp_sales)

mp_sales.save('KC_map_sales.html')
mp_sales

<div class="alert alert-block alert-info">On the basis of the choropleth map above:    
<ol>
<li> Zipcodes 98042 and 98038 have seen a lot of sales. So have zipcodes 98117, 98103 and 98115. It would make sense to target these areas if the sales strategy was to follow a volume strategy and complete as many sales as possible.</li>
<li> Zipcode 98039 stands out as the zipcode with the highest median price, but it may not be the area that would present the best target in a value strategy.</li>
<li> Zipcodes 98004, 98006 and 98052 are the locations to target if we want to follow a value strategy and aim to complete fewer sales, but where sales are completed they are deals of a higher value (and therefore bring in more commission).</li>
    </ol>
</div>

## In which month is the most property sold in King County?

Because we only have price data for only 2 years, it is impossible to draw any firm conclusions as to any overall pattern. But from the perspective of the firm, which would be the month with the highest volume of sales (and therefore the **worst time in the year to take a holiday**)?

In [ ]:
# sum up all the sales by value of property in King County by a particular date
# create a dataframe price_time which has, as each row, a unique date and on such date the total value of sales in King County 
price_time = pd.DataFrame(raw_data.groupby('date')['price'].sum()) 

# add an additional column which assigns a month to each unique data
# for instance all sales occurring on the date 2014-05-07 would have "5" in this column
price_time['month']=price_time.index.month

# let's group all the sales from the same month together
# this will help us identify any seasonality in the data - in which months do property sell well?
price_month = pd.DataFrame(price_time.groupby('month')['price'].average())

# let's express the sales per month per million. 
# this can help us answer the question, in which month was the most property sold in 2014 to 2015?
price_month['price'].apply(lambda x: x/10**6).sort_values(ascending=True)


Representing the information above graphically below:

In [ ]:
#define the figure
fig,ax = plt.subplots()

# use a white background and other settings 
plt.rcParams['axes.facecolor'] = 'white'
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.tick_params(axis='x', colors='black', pad =.2)
ax.grid(color='grey', linestyle='-.', linewidth=1, alpha=0.4)

# specify colors
# colour list and import package
from itertools import cycle, islice
my_colors = 'rgbkymc'

# plot data in price_month
price_month.plot(kind='barh', figsize=(12,10), color = my_colors, width = .7, alpha = .7, edgecolor = 'blue', ax=ax)
    
# labelling around the graph
plt.xlabel('Value of Sales (per USD1,000m)', fontsize = 15)
plt.ylabel('Month', fontsize = 15)
plt.title('Value of Property Sales achieved in King County by calendar month', fontsize = 18, loc ='center')
plt.tight_layout()
plt.show()

The worst month for an estate agent to take a holiday is in May and the best month to do so is in January. In fact, it looks like the warmer months of the year sees more sales, but we would need price data from more years to be sure of this conclusion.

# Conclusion

<div class="alert alert-block alert-success">
<b>In conclusion</b> we have:
    <ol>
        <li> developed and fit a model around <b>zipcode</b>, <b>grade</b> and <b>sqft_living</b> which will predict the sales price of any given property in King County with an r-squared of <b>0.734</b>;</li>
        <li> based on this model, written a simple function such that provided with the <b>zipcode</b>, <b>grade</b> and <b>sqft_living</b> of any given property in King County, our firm of estate agents will be able to generate a benchmark price upon which negotiations between buyers and sellers can proceed;</li>
        <li> using <b>choropleth maps</b>, our firm of estate agents have learnt more about the real estate market in King County by zipcode. In particular, this firm of new entrants to the market know which zipcode contain older housing stock and higher quality housing; </li>
        <li> using the same choropleth maps, the firm of estate agents can work out which zipcodes to target given two different sales strategies they may wish to execute; and</li>
        <li> last but not least, the firm has a rough idea (given only 2 years data) of when it might be a good time to take a holiday!</li>
            </ol>
</div>